In [1]:

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))



/kaggle/input/churn-detection/train.csv
/kaggle/input/churn-detection/test.csv


In [3]:
train_df = pd.read_csv('/kaggle/input/churn-detection/train.csv')
test_df = pd.read_csv('/kaggle/input/churn-detection/test.csv')

In [4]:
import pandas as pd
from sklearn.preprocessing import LabelEncoder


# Convert 'TotalCharges' to numeric
train_df['TotalCharges'] = pd.to_numeric(train_df['TotalCharges'], errors='coerce')
test_df['TotalCharges'] = pd.to_numeric(test_df['TotalCharges'], errors='coerce')



In [6]:
# Fill missing values with the median
train_df['TotalCharges'] = train_df['TotalCharges'].fillna(train_df['TotalCharges'].median())
test_df['TotalCharges'] = test_df['TotalCharges'].fillna(test_df['TotalCharges'].median())


In [7]:
# Drop 'id' column
train_df.drop(columns=['id'], inplace=True)
test_ids = test_df['id']
test_df.drop(columns=['id'], inplace=True)

In [8]:
# Encode categorical variables
categorical_columns = train_df.select_dtypes(include=['object']).columns.tolist()
categorical_columns.remove('Churn')

In [9]:
# Encode using LabelEncoder
label_encoders = {}
for col in categorical_columns:
    le = LabelEncoder()
    train_df[col] = le.fit_transform(train_df[col])
    test_df[col] = le.transform(test_df[col])
    label_encoders[col] = le

In [10]:
# Encode target
target_encoder = LabelEncoder()
train_df['Churn'] = target_encoder.fit_transform(train_df['Churn'])

In [11]:
train_df.head()

,gender,SeniorCitizen,Partner,Dependents,tenure,PhoneService,MultipleLines,InternetService,OnlineSecurity,OnlineBackup,DeviceProtection,TechSupport,StreamingTV,StreamingMovies,Contract,PaperlessBilling,PaymentMethod,MonthlyCharges,TotalCharges,Churn
0,1,0,1,0,51,1,2,0,0,0,0,2,2,2,2,1,0,75.20,3901.25,0
1,0,0,0,0,11,1,0,0,2,0,0,2,0,0,1,1,3,54.60,617.85,0
2,0,0,0,0,16,1,0,2,1,1,1,1,1,1,1,0,0,20.10,318.60,0
3,0,0,0,0,34,1,2,0,0,0,0,2,2,0,0,0,3,65.00,2157.50,0
4,0,1,1,0,43,1,2,1,0,0,2,2,2,2,1,1,2,107.55,4533.90,1


In [12]:
import pandas as pd
from sklearn.preprocessing import LabelEncoder

In [13]:
train_df = pd.read_csv('/kaggle/input/churn-detection/train.csv')
test_df = pd.read_csv('/kaggle/input/churn-detection/test.csv')

In [14]:
train_df['TotalCharges'] = pd.to_numeric(train_df['TotalCharges'], errors='coerce')
test_df['TotalCharges'] = pd.to_numeric(test_df['TotalCharges'], errors='coerce')

In [16]:
train_df['TotalCharges'] = train_df['TotalCharges'].fillna(train_df['TotalCharges'].median())
test_df['TotalCharges'] = test_df['TotalCharges'].fillna(test_df['TotalCharges'].median())


In [17]:
train_df.drop(columns=['id'], inplace=True)
test_ids = test_df['id']
test_df.drop(columns=['id'], inplace=True)

In [18]:
#Encode categorical variables
categorical_columns = train_df.select_dtypes(include=['object']).columns.tolist()
categorical_columns.remove('Churn')


In [19]:
# Encode using LabelEncoder
label_encoders = {}
for col in categorical_columns:
    le = LabelEncoder()
    train_df[col] = le.fit_transform(train_df[col])
    test_df[col] = le.transform(test_df[col])
    label_encoders[col] = le

In [20]:
# Encode target
target_encoder = LabelEncoder()
train_df['Churn'] = target_encoder.fit_transform(train_df['Churn'])


In [21]:
# Show cleaned and encoded data
train_df.head()

,gender,SeniorCitizen,Partner,Dependents,tenure,PhoneService,MultipleLines,InternetService,OnlineSecurity,OnlineBackup,DeviceProtection,TechSupport,StreamingTV,StreamingMovies,Contract,PaperlessBilling,PaymentMethod,MonthlyCharges,TotalCharges,Churn
0,1,0,1,0,51,1,2,0,0,0,0,2,2,2,2,1,0,75.20,3901.25,0
1,0,0,0,0,11,1,0,0,2,0,0,2,0,0,1,1,3,54.60,617.85,0
2,0,0,0,0,16,1,0,2,1,1,1,1,1,1,1,0,0,20.10,318.60,0
3,0,0,0,0,34,1,2,0,0,0,0,2,2,0,0,0,3,65.00,2157.50,0
4,0,1,1,0,43,1,2,1,0,0,2,2,2,2,1,1,2,107.55,4533.90,1


In [22]:
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, accuracy_score


In [24]:
#Split the training data
X = train_df.drop(columns=['Churn'])
y = train_df['Churn']

In [25]:
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

In [26]:
 #Train a Random Forest Classifier
model = RandomForestClassifier(random_state=42)
model.fit(X_train, y_train)

RandomForestClassifier(random_state=42)

In [28]:
# Evaluate on validation set
y_pred = model.predict(X_val)
accuracy = accuracy_score(y_val, y_pred)
report = classification_report(y_val, y_pred, target_names=target_encoder.classes_)

In [31]:
accuracy, report

(0.7985803016858918,
 '              precision    recall  f1-score   support\n\n          No       0.83      0.92      0.87       836\n         Yes       0.67      0.44      0.53       291\n\n    accuracy                           0.80      1127\n   macro avg       0.75      0.68      0.70      1127\nweighted avg       0.78      0.80      0.78      1127\n')